In [1]:
import requests
import json

In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
# Generic Function to send GET requests to DraCor API

#corpusname:str -> []
def get(**kwargs):
    #corpusname=corpusname
    #playname=playname
    #apibase="https://dracor.org/api/"
    #method=method
    #parse_json: True
    
    #could set different apibase, e.g. https://staging.dracor.org/api/ [not recommended, pls use the production server]
    if "apibase" in kwargs:
        if kwargs["apibase"].endswith("/"):
            apibase = kwargs["apibase"]
        else:
            apibase = kwargs["apibase"] + "/"
    else:
        #use local API per default
        apibase = "http://dracor.org/api/"
    if "corpusname" in kwargs and "playname" in kwargs:
        # used for /api/corpora/{corpusname}/play/{playname}/
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/play/" + kwargs["playname"]
    elif "corpusname" in kwargs and not "playname" in kwargs:
        if "method" in kwargs:
            request_url = apibase + "corpora/" + kwargs["corpusname"] + "/" + kwargs["method"]
        else:
            request_url = apibase + "corpora/" + kwargs["corpusname"] 
    elif "method" in kwargs and not "corpusname" in kwargs and not "playname" in kwargs:
            request_url = apibase + kwargs["method"]
            
    else: 
        #nothing set
        request = request_url = apibase + "info"
    
    #send the response
    r = requests.get(request_url)
    if r.status_code == 200:
        #success!
        if "parse_json" in kwargs:
            if kwargs["parse_json"] == True:
                json_data = json.loads(r.text)
                return json_data
            else:
                return r.text
        else:
            return r.text
    else:
        raise Exception("Request was not successful. Server returned status code: "  + str(r.status_code))

In [4]:
from dracor_api_response_schemas import is_valid

## /info

In [5]:
from dracor_api_response_schemas import InfoSchema

In [6]:
info_response = get(method="info",parse_json=True)

In [7]:
info_response

{'name': 'DraCor API',
 'version': '0.89.0',
 'status': 'beta',
 'existdb': '6.0.1'}

In [8]:
# Validate against the schema
# In case of an validation Error (returned value is False), set the flag "catch_error=False", which will return the 
# message from the exception
is_valid(info_response, InfoSchema)

INFO:root:Validation passed.


True

## /corpora

In [9]:
from dracor_api_response_schemas import CorpusInCorporaSchema

In [10]:
corpora_response_no_metrics = get(method="corpora", parse_json=True)

In [11]:
print("Validation of /corpora without metrics included:")
for corpus in corpora_response_no_metrics:
    if is_valid(corpus, CorpusInCorporaSchema):
        print("Data of corpus " + corpus["title"] + " passed validation.")
    else:
        print ("ERROR " + corpus["title"])

INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.


Validation of /corpora without metrics included:
Data of corpus Alsatian Drama Corpus passed validation.
Data of corpus Bashkir Drama Corpus passed validation.
Data of corpus Calderón Drama Corpus passed validation.
Data of corpus French Drama Corpus passed validation.
Data of corpus German Drama Corpus passed validation.
Data of corpus German Shakespeare Drama Corpus passed validation.
Data of corpus Greek Drama Corpus passed validation.
Data of corpus Hungarian Drama Corpus passed validation.
Data of corpus Italian Drama Corpus passed validation.
Data of corpus Roman Drama Corpus passed validation.
Data of corpus Russian Drama Corpus passed validation.
Data of corpus Shakespeare Drama Corpus passed validation.
Data of corpus Spanish Drama Corpus passed validation.
Data of corpus Swedish Drama Corpus passed validation.
Data of corpus Tatar Drama Corpus passed validation.


In [12]:
corpora_response_incl_metrics = get(method="corpora?include=metrics", parse_json=True)

In [13]:
print("Validation of /corpora with metrics included:")
for corpus in corpora_response_incl_metrics:
    if is_valid(corpus, CorpusInCorporaSchema):
        print("Data of corpus " + corpus["title"] + " passed validation.")
    else:
        print ("ERROR " + corpus["title"])

INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.
INFO:root:Validation passed.


Validation of /corpora with metrics included:
Data of corpus Alsatian Drama Corpus passed validation.
Data of corpus Bashkir Drama Corpus passed validation.
Data of corpus Calderón Drama Corpus passed validation.
Data of corpus French Drama Corpus passed validation.
Data of corpus German Drama Corpus passed validation.
Data of corpus German Shakespeare Drama Corpus passed validation.
Data of corpus Greek Drama Corpus passed validation.
Data of corpus Hungarian Drama Corpus passed validation.
Data of corpus Italian Drama Corpus passed validation.
Data of corpus Roman Drama Corpus passed validation.
Data of corpus Russian Drama Corpus passed validation.
Data of corpus Shakespeare Drama Corpus passed validation.
Data of corpus Spanish Drama Corpus passed validation.
Data of corpus Swedish Drama Corpus passed validation.
Data of corpus Tatar Drama Corpus passed validation.


## /corpora/{corpusname} 

In [14]:
from dracor_api_response_schemas import CorpusSchema

In [15]:
corpus_response = get(corpusname="ger", parse_json=True)

In [16]:
#corpus_response

In [17]:
# is_valid(corpus_response, CorpusSchema, catch_error=False)
is_valid(corpus_response, CorpusSchema)

INFO:root:Validation passed.


True

In [18]:
#Validate all
corpusnames = []
for item in corpora_response_no_metrics:
    corpusnames.append(item["name"])

In [19]:
corpusnames

['als',
 'bash',
 'cal',
 'fre',
 'ger',
 'gersh',
 'greek',
 'hun',
 'ita',
 'rom',
 'rus',
 'shake',
 'span',
 'swe',
 'tat']

for corpusname in corpusnames:
    corpus_response = get(corpusname=corpusname, parse_json=True)
    print(f"Validating {corpusname}.")
    is_valid(corpus_response, CorpusSchema, catch_error=False)

## /corpora/{corpusname}/metadata

In [20]:
from dracor_api_response_schemas import PlayMetadataSchema

In [21]:
# metadata_plays_of_corpus_response = get(method="metadata",corpusname="ger",parse_json=True)

In [22]:
# metadata_plays_of_corpus_response[0]

In [23]:
#is_valid(metadata_plays_of_corpus_response[0], PlayMetadataSchema, catch_error=False)

for play in metadata_plays_of_corpus_response:
    is_valid(play, PlayMetadataSchema, catch_error=True)  

# Validate all
for corpusname in corpusnames:
    print(f"Validating corpus {corpusname}.")
    plays_metadata = get(method="metadata",corpusname=corpusname,parse_json=True)
    
    for play in plays_metadata:
        print(f"Validating {play['id']}.")
        is_valid(play, PlayMetadataSchema, catch_error=False)  
    

OK, all valid... to test, change the cell above back from markdown to code

## /corpora/{corpusname}/play/{playname}

In [24]:
from dracor_api_response_schemas import PlaySchema

In [25]:
def validate_play(corpusname: str = None, playname:str = None, catch_error = True):
    data = get(corpusname=corpusname, playname=playname,parse_json=True)
    return is_valid(data, PlaySchema, catch_error=catch_error) 
    

In [26]:
#validate_play(corpusname="ger", playname="lessing-emilia-galotti")

In [27]:
corpusnames_no_cal = corpusnames
corpusnames_no_cal.remove("cal")

for corpusname in corpusnames_no_cal:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

# do it separately..
# UNKWON ... stop at first stupid error
for corpusname in ["hun"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["ita"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["rom"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["rus"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

"""    {
      "id": "Demetrius_MND",
      "name": "Demetrius",
      "isGroup": false,
      "sex": "MAE"
    },

"""
for corpusname in ["shake"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["span"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["swe","tat"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["tat"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play(corpusname=corpusname, playname=playname, catch_error=False)

## /corpora/{corpusname}/play/{playname}/metrics

In [28]:
from dracor_api_response_schemas import PlayMetricsSchema

In [29]:
def validate_play_metrics(corpusname: str = None, playname:str = None, catch_error = True):
    data = get(corpusname=corpusname, method="metrics", playname=playname,parse_json=True)
    return is_valid(data, PlayMetricsSchema, catch_error=catch_error) 

In [30]:
validate_play_metrics(corpusname="ger", playname="lessing-emilia-galotti")

INFO:root:Validation passed.


True

for corpusname in ["ger"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play_metrics(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in corpusnames_no_cal:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_play_metrics(corpusname=corpusname, playname=playname, catch_error=False)

## /corpora/{corpusname}/play/{playname}/cast

In [31]:
from dracor_api_response_schemas import CastItemSchema

In [32]:
def validate_cast_items(corpusname: str = None, playname:str = None, catch_error = False):
    data = get(corpusname=corpusname, method="cast", playname=playname,parse_json=True)
    valid_flag = True
    for item in data:
        if is_valid(item, CastItemSchema, catch_error=catch_error) == False:
            valid_flag = False
            print(f"Error in {item['id']}")
    return valid_flag
        

In [33]:
validate_cast_items(corpusname="ger", playname="lessing-emilia-galotti")

True

for corpusname in corpusnames:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["cal"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["ger","rus"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["rus"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["greek", "rom", "ita", "span"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["ita", "span"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

for corpusname in ["shake"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

In [37]:
for corpusname in ["gersh"]:
    #print(corpusname)
    plays = get(corpusname=corpusname,parse_json=True)["dramas"]
    playnames = []
    for play in plays:
        playnames.append(play["name"])
    #print(playnames)
    
    for playname in playnames:
        print(f"Validating {corpusname} – {playname}.")
        validate_cast_items(corpusname=corpusname, playname=playname, catch_error=False)

Validating gersh – antonius-und-cleopatra.
Validating gersh – coriolanus.
Validating gersh – cymbeline.
Validating gersh – das-wintermaerchen.
Validating gersh – der-kaufmann-von-venedig.
Validating gersh – der-sturm.
Validating gersh – der-widerspenstigen-zaehmung.
Validating gersh – die-beiden-edlen-vettern.
Validating gersh – die-beiden-veroneser.
Validating gersh – die-komoedie-der-irrungen.
Validating gersh – die-lustigen-weiber-von-windsor.
Validating gersh – ein-sommernachtstraum.
Validating gersh – ende-gut-alles-gut.
Validating gersh – hamlet-prinz-von-daenemark.
Validating gersh – julius-caesar.
Validating gersh – koenig-heinrich-4-teil-1.
Validating gersh – koenig-heinrich-4-teil-2.
Validating gersh – koenig-heinrich-5.
Validating gersh – koenig-heinrich-6-teil-1.
Validating gersh – koenig-heinrich-6-teil-2.
Validating gersh – koenig-heinrich-6-teil-3.
Validating gersh – koenig-heinrich-8.
Validating gersh – koenig-johann.
Validating gersh – koenig-lear.
Validating gersh – k